In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection, metrics   #Additional scklearn functions

In [4]:
data = pd.read_csv('./training_data_v3.csv', low_memory=False)

In [5]:
data.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar36,mvar37,mvar38,mvar39,mvar42,mvar43,mvar44,mvar46,mvar47,default_ind
0,0.000000,0.444201,0.045909,0.000000,0.0,0.0,0.000000,0.009766,0.001103,0.080936,...,0.106061,0.166667,0.042105,0.066667,0.043480,0.109890,0.615677,0.0,1.0,0.0
1,0.000013,0.772429,0.022467,0.000000,0.0,0.0,0.002927,0.012230,0.010867,0.036558,...,0.106061,0.000000,0.021053,0.000000,0.000000,0.142857,0.615006,0.0,0.0,1.0
2,0.000038,0.538293,0.006087,0.000000,0.0,0.0,0.056874,0.042930,0.016980,0.040732,...,0.030303,0.050000,0.021053,0.000000,0.076925,0.032967,0.502214,0.0,0.0,0.0
3,0.000050,0.643326,0.000328,0.001297,0.0,0.0,0.156418,0.008921,0.018827,0.016013,...,0.015152,0.050000,0.021053,0.000000,0.000000,0.010989,0.921913,0.0,0.0,0.0
4,0.000100,0.750547,0.003769,0.000000,0.0,0.0,0.009957,0.063265,0.061095,0.141805,...,0.075758,0.200000,0.021053,0.000000,0.000000,0.109890,0.892137,0.0,0.0,0.0


In [6]:
X = data[data.loc[:, data.columns != 'default_ind'].columns]
Y = data['default_ind']
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=.2, random_state=1)

In [7]:
X_train['default_ind'] = Y_train

/home/akul/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
X_train.head()
target = 'default_ind'

In [9]:
np.sum(X_train['default_ind'].values)

10074.0

In [10]:
np.sum(Y_train.values)

10074.0

In [11]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        #xgb_param['num_class']=2
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
             early_stopping_rounds=early_stopping_rounds,stratified=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))

In [12]:
predictors = [x for x in X_train.columns if x not in ['default_ind']]
print(predictors)

['application_key', 'mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6', 'mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar12', 'mvar13', 'mvar14', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar21', 'mvar24', 'mvar25', 'mvar26', 'mvar27', 'mvar28', 'mvar29', 'mvar32', 'mvar33', 'mvar34', 'mvar36', 'mvar37', 'mvar38', 'mvar39', 'mvar42', 'mvar43', 'mvar44', 'mvar46', 'mvar47']


In [14]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:hinge',
 nthread=4,
 scale_pos_weight=1,
 seed=37)
modelfit(xgb1, X_train, predictors)


Model Report
Accuracy : 0.8393


In [15]:
Y_pred=xgb1.predict(X_test)

In [16]:
xgb1.predict_proba(X_test)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [17]:
float(np.sum(Y_pred == Y_test))/float(np.sum(Y_test == Y_test))

0.8286287089013633

In [18]:
metrics.accuracy_score(Y_test, Y_pred)

0.8286287089013633

In [19]:
X_lead = pd.read_csv('./RefinedData/refined_leaderboard_data.csv', low_memory=False)

In [20]:
X_lead.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar34,mvar36,mvar37,mvar38,mvar39,mvar42,mvar43,mvar44,mvar46,mvar47
0,350053.0,1753.0,0.5001,0.000,0.0,0.0,206.000000,19179.000000,206.000000,14221.000000,...,0.0,3.0,11.0,1.0,0.000000,0.42105,3.0,0.856610,0.000000,0.0
1,350054.0,1894.0,1.9701,0.000,0.0,0.0,297.000000,19820.000000,19820.000000,176895.000000,...,0.0,3.0,12.0,3.0,0.000000,0.00000,15.0,0.943910,0.000000,1.0
2,350055.0,1774.0,0.1718,0.000,0.0,0.0,197.000000,2563.000000,877.000000,8869.000000,...,0.0,3.0,4.0,1.0,0.000000,0.33333,3.0,0.764670,0.000000,1.0
3,350056.0,1586.0,0.1123,5.299,0.0,0.0,1624.069192,17750.368864,6753.202313,33825.554662,...,0.0,1.0,0.0,1.0,0.089912,1.50000,0.0,0.702558,0.092712,1.0
4,350057.0,1832.0,1.4442,0.000,0.0,0.0,179.000000,7577.000000,179.000000,21059.000000,...,0.0,17.0,0.0,3.0,0.000000,0.00000,23.0,0.791900,0.000000,0.0


In [21]:
Y_lead=pd.DataFrame(xgb1.predict(X_lead))

In [22]:
Y_prob = xgb1.predict_proba(X_lead)
Y_prob

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [23]:
Y_wt = Y_prob[:,0]
Y_wt

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [24]:
sol = pd.DataFrame()
sol['priority'] = pd.Series(Y_wt)
sol['application_key'] = X_lead['application_key'].astype(dtype='int')
sol['default_ind'] = Y_lead.astype(dtype='int')

In [25]:
sol.head()

,priority,application_key,default_ind
0,1.0,350053,0
1,1.0,350054,0
2,1.0,350055,0
3,0.0,350056,1
4,1.0,350057,0


In [26]:
sol.sort_values(by = 'priority', ascending = False)

,priority,application_key,default_ind
0,1.0,350053,0
15051,1.0,365104,0
15062,1.0,365115,0
15060,1.0,365113,0
15059,1.0,365112,0
15057,1.0,365110,0
15055,1.0,365108,0
15054,1.0,365107,0
15053,1.0,365106,0
15050,1.0,365103,0


In [75]:
sol = sol.drop('priority', axis = 1)
sol.to_csv(path_or_buf='./Submissions/Analyzing_IITGuwahati_32.csv', index = False, header=False)